In [17]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [12]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-4o-mini",
    prompt="What is 2+2?",
    max_tokens=50
)

print(response.choices[0].text.strip())

** (a basic math question)

**What is the capital of France?** (a general knowledge question)

Please let me know which topic you would like to explore further or feel free to ask anything else!


In [13]:
from google import genai

gemini_api_key = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=gemini_api_key)

messages = ["What is 2+2?"]
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=messages
)

print(response.text)

2 + 2 = 4



In [14]:
deepseek_api_key=os.getenv("DEEPSEEK_API_KEY")

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

message = [{'role':'user','content':"what is 2+3?"}]

response = deepseek.chat.completions.create(model=model_name, messages=message)
print(response.choices[0].message.content)





The sum of 2 and 3 is **5**.  

\[ 2 + 3 = 5 \]  

Let me know if you'd like help with anything else! 😊


In [ ]:
from anthropic import Anthropic

import os
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
print(anthropic_api_key)

# client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# # Use the messages API instead of completions
# response = client.messages.create(
#     model="claude-3-sonnet-20240229",  # Use a current model
#     max_tokens=50,
#     messages=[
#         {"role": "user", "content": "What is 2+2?"}
#     ]
# )

# print(response.content[0].text.strip())

/var/folders/h_/vs1_c0bd0kn_fms0n071g0gr0000gn/T/ipykernel_42728/735814247.py:6: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

In [8]:
!pip3 install anthropic

  Using cached anthropic-0.52.2-py3-none-any.whl.metadata (25 kB)
Using cached anthropic-0.52.2-py3-none-any.whl (286 kB)


In [ ]:
from huggingface_hub import InferenceClient
client = InferenceClient("meta-llama/Llama-2-7b-chat-hf")
response = client.text_generation(
    "What is 2+2?",
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2,
    return_full_text=False
)
print(response[0]['generated_text'].strip())  

In [9]:
from agents import Agent, Runner, trace

# Make an agent with name, instructions, model
agent = Agent(name="Jokester", instructions="You are a joke teller", model="gpt-4o-mini")
with trace("Telling a joke"):
    result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
    print(result.final_output)

Why did the autonomous AI agent break up with its partner?

Because it couldn't handle all the emotional data!
